In [17]:
print("Hello World")

Hello World


In [18]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-25 00:02:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Connecting to 127.0.0.1:7897... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.53MB/s    in 0.7s    

2026-02-25 00:02:51 (1.53 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(set(text))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


def encode(x):
    return [stoi[ch] for ch in x]


def decode(x):
    return "".join([itos[i] for i in x])


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + 1 + block_size] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is tensor([24]) the target: 43
When input is tensor([24, 43]) the target: 58
When input is tensor([24, 43, 58]) the target: 5
When input is tensor([24, 43, 58,  5]) the target: 57
When input is tensor([24, 43, 58,  5, 57]) the target: 1
When input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
When input is tensor([44]) the target: 53
When input is tensor([44, 53]) the target: 56
When input is tensor([44, 53, 56]) the target: 1
When input is tenso

In [11]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are (B,T)

        # logits is (B,T,C)
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(
                logits.view(B * T, C),
                targets.view(B * T),
            )

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx -> context in batch
        # (B,T)

        for _ in range(max_new_tokens):
            # logits -> scores (B,T,C)
            logits, _ = self(idx)

            # pick last, become (B,C)
            logits = logits[:, -1, :]

            # softmax -> probabilities
            # (B,C)
            probs = F.softmax(logits, dim=-1)

            # sample
            # idx_next: (B,1)
            idx_next = torch.multinomial(probs, num_samples=1)

            # cat (B,T) (B,1)
            # get (B,T+1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


m = BigramLanguageModel(vocab_size)
# print("xb", xb.shape)
# print("logits", m.token_embedding_table(xb).shape)
logits, loss = m(xb, yb)
print("yb", yb.shape)
print(logits.shape)
print(loss)


yb torch.Size([4, 8])
torch.Size([4, 8, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [14]:
print(
    decode(
        m.generate(
            torch.zeros(
                (1, 1),
                dtype=torch.long,
            ),
            max_new_tokens=100,
        )[0].tolist()
    )
)


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [15]:
# optim
optimizer = torch.optim.AdamW(
    m.parameters(),
    lr=1e-3,
)


In [ ]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())


3.6824676990509033
3.691699504852295
3.775355100631714
3.567988157272339
3.6252493858337402
3.6441354751586914
3.6750290393829346
3.7602500915527344
3.655397653579712
3.651463270187378
3.688493490219116
3.702279806137085
3.613967180252075
3.7137131690979004
3.7722153663635254
3.596604585647583
3.5304887294769287
3.6604740619659424
3.5630905628204346
3.59187388420105
3.5366814136505127
3.5414323806762695
3.608551025390625
3.7356154918670654
3.6624581813812256
3.6300947666168213
3.6306936740875244
3.60614275932312
3.5736145973205566
3.67541766166687
3.5313475131988525
3.6371285915374756
3.661085844039917
3.6264193058013916
3.650089979171753
3.53578782081604
3.6060996055603027
3.6430013179779053
3.591032028198242
3.6671528816223145
3.5971174240112305
3.654073476791382
3.67277455329895
3.5566089153289795
3.72536039352417
3.5462849140167236
3.59515380859375
3.6235764026641846
3.646811008453369
3.632132053375244
3.59301495552063
3.6381375789642334
3.583293914794922
3.7322165966033936
3.73939

In [20]:
print(
    decode(
        m.generate(
            torch.zeros(
                (1, 1),
                dtype=torch.long,
            ),
            max_new_tokens=300,
        )[0].tolist()
    )
)


mactore het e s f lazL:
NUTheishal atot  mes tove's pe ciond r jenge paris otho hear bundiqbuser ss:
Thargray s t e thing d isu lds
ENNE the tupowa!
HAsst y, hr'ste mire
Sichiorendout;
AUTor.
OR:
Delbyounder:
Nais$gher g f thefr!
The haras INoleert yord So ben. swe tUMocceane t senoure aimaringe and
